# 15.4.6 Seq+Seq2Vec con atención

## Textual entailment o inferencia en lenguaje natural

Dados dos textos, una premisa y una hipótesis, decimos que
* la premisa **implica (entails)** la hipótesis si la hipótesis es verosímil dada la premisa
* la premisa **contradice** la hipótesis si la hipótesis no es verosímil dada la premisa
* la relación entre ambas es **neutra** si la hipótesis puede darse o no, dada la premisa

**Stanford Natural Language Inference (SNLI):** $\;$ 550K pares de frases etiquetados según las tres categorías anteriores

## Modelo de 2016 para textual entailment

**Representación:** $\;$ clasificación de $\,$ `I do not sleep` $\,$ **implica** $\,$ `I am tired`
<div><table align=center><tr>
<td style="border: none;"><img src="Figure_15.21.png" width=600/></td>
</tr></table></div>

**Entrada:** $\;$ premisa $\,\mathbf{A}=(\boldsymbol{a}_1,\dotsc,\boldsymbol{a}_m)\,$ e hipótesis $\,\mathbf{B}=(\boldsymbol{b}_1,\dotsc,\boldsymbol{b}_n)$; $\;\boldsymbol{a}_i,\boldsymbol{b}_j\in\mathbb{R}^E$ son vectores de embedding de palabras

**Paso 1:** $\;$ comparación de la premisa con la hipótesis

Cada $\,\boldsymbol{a}_i\,$ atiende a cada $\,\boldsymbol{b}_j\,$ para calcular un peso de atención con un MLP $\,f:\mathbb{R}^E\to\mathbb{R}^D,$ 
$$e_{ij}=f(\boldsymbol{a}_i)^tf(\boldsymbol{b}_j)$$ 
y obtener un promedio ponderado de las palabras que encajan en la hipótesis
$$\boldsymbol{\beta}_i=\sum_{j=1}^n\frac{\exp(e_{ij})}{\sum_{k=1}^m\exp(e_{ik})}\boldsymbol{b}_j$$

Luego, se compara $\boldsymbol{a}_i$ con $\boldsymbol{\beta}_i$ mediante transformación de su concatenación a un espacio oculto con un MLP $g:\mathbb{R}^{2E}\to\mathbb{R}^H$
$$\boldsymbol{v}_{A,i}=g([\boldsymbol{a}_i,\boldsymbol{\beta}_i])\qquad(i=1,\dotsc,m)$$
y se suman las comparaciones para obtener una similitud global de la premisa a la hipótesis:
$$\boldsymbol{v}_A=\sum\nolimits_{i=1}^m\boldsymbol{v}_{A,i}$$

**Paso 2:** $\;$ comparación de la hipótesis con la premisa de forma análoga
$$\begin{align}
  \boldsymbol{\alpha}_j&=\sum_{i=1}^n\frac{\exp(e_{ij})}{\sum_{k=1}^m\exp(e_{kj})}\boldsymbol{a}_i\\%
  \boldsymbol{v}_{B,j}&=g([\boldsymbol{b}_j,\boldsymbol{\alpha}_j])\qquad(j=1,\dotsc,n)\\%
  \boldsymbol{v}_B&=\sum_{j=1}^n\boldsymbol{v}_{B,j}%
\end{align}$$

**Paso 3:** $\;$ se clasifica la salida con otro MLP, $\,h:\mathbb{R}^{2H}\to\mathbb{R}^3$
$$\hat{y}=h([\boldsymbol{v}_A,\boldsymbol{v}_B])$$

## Semantic textual similarity

Tarea relacionada; se trata de predecir el grado de similitud semántica entre dos frases; p.e., en el **benchmark STS** se mide de 0 a 5